# 第七讲：最优间隔分类器、支持向量机

简单的回顾上一讲的内容：

* 我们定义了新的假设函数：

    $$\begin{align}h_{w,b}&=g\left(w^Tx+b\right)\\g(z)&=\begin{cases}1 &z\gt 0\\-1 &z\lt 0\end{cases}\\y&=\{-1,1\}\end{align}$$

* 给出了函数间隔的定义：

    $$\hat{\gamma}^{(i)}=y^{(i)}\left(w^Tx^{(i)}+b\right)$$

* 以及几何间隔的定义：

    $$\gamma^{(i)}=y^{(i)}\left(\left(\frac{w}{\lVert w\rVert}\right)^Tx^{(i)}+\frac{b}{\lVert w\rVert}\right)$$

* 了解了函数间隔与几何间隔的关系：

    $$\gamma=\frac{\hat\gamma}{\lVert w\rVert}$$

* 同时也了解了其几何意义：由$w^Tx+b=0$确定的分类超平面将正负样本分隔开，同时使超平面与样本间的最小间隔尽可能大。我们也发现，如果按比例缩放参数$(w,b)$，并不会对假设结果造成任何影响，因为给参数同时乘以一个系数并不会影响超平面的位置。

接着上一讲，我们继续介绍支持向量机的相关知识。

## 4. 最优间隔分类器（optimal margin classifier）

通过上一讲的介绍，对于给定的训练集，我们会自然的想要找到一个能够使（几何）间隔达到最大的判定边界，因为这样的判定边界能够得到高度可信的预测集，同时也是对训练集很好的拟合。这也将帮助我们得到一个将正负样本分隔（几何间隔）开的分类器。

目前为止，我们都假设训练集是线性可分的，即我们一定可以找到一个能够将正负样本分开的分类超平面。那么，我们怎样找到将几何间隔最大化的分类超平面呢？讨论下面的优化问题：

$$\begin{align}\displaystyle\operatorname*{max}_{w,b}&\quad\gamma\\\mathrm{s.t.}&\quad y^{(i)}\left(w^Tx^{(i)}+b\right)\geq\gamma,\quad i=1,\cdots,m\\&\quad\lVert w\rVert=1\end{align}$$

上式表示，在满足每一个训练样本的函数间隔都不小于$\gamma$的情况下，使用$(w,b)$构造$\gamma$的能够取到的最大值，同时有限制条件$\lVert w\rVert=1$保证函数间隔等于几何间隔。通过求得的$(w,b)$就可以计算出该训练集的最大几何间隔。（注：$\mathrm{s.t.}$是“subject to”的缩写，意为“受限于”。）

上面的优化问题有个棘手的限制条件$\lVert w\rVert=1$（这是一个糟糕的非凸性约束，即参数$w$可能位于一个单位圆环/球面上），这显然不是可以提交给标准的凸优化软件（如梯度下降法、牛顿法等）去解决的问题。所以，我们将上面的问题变形为另一种的形式：

$$\begin{align}\displaystyle\operatorname*{max}_{w,b}&\quad\frac{\hat\gamma}{\lVert w\rVert}\\\mathrm{s.t.}&\quad y^{(i)}\left(w^Tx^{(i)}+b\right)\geq\hat\gamma,\quad i=1,\cdots,m\end{align}$$

也就是我们选择最大化几何间隔$\frac{\hat\gamma}{\lVert w\rVert}$，同时满足所有样本的函数间隔不小于$\hat\gamma$的条件。因为函数间隔和几何间隔的关系为$\gamma=\frac{\hat\gamma}{\lVert w\rVert}$，而又舍弃了限制条件$\lVert w\rVert=1$，所以这次我们觉得应该可以求出最大值了。不过值得注意的是，这次最大化的对象$\frac{\hat\gamma}{\lVert w\rVert}$是非凸的，而我们同样没有现成的凸优化软件来解决此类问题。

于是，我们发现在第一个优化问题中，存在非凸性的限制条件，而在第二个优化问题中，存在非凸性的优化目标，所以我们并不能保证软件可以找到全局最小值。（要带上限制条件$\lVert w\rVert=1$是因为我们的优化想要度量的是几何间隔）。

回想前一讲，我们知道，按比例缩放参数$(w,b)$对假设结果没有任何影响，我们现在可以利用这一点。我们现在来引入限制条件：对于给定的训练集，以$(w,b)$为参数的函数间隔必须为$1$：

$$\hat\gamma=1$$

给参数$w,b$乘以缩放常量会使得函数间隔缩放同样的倍数，则可以通过缩放$w,b$来满足上面的限制条件。将这一限制条件加入上面的假设中，于是问题变为最大化$\frac{\hat\gamma}{\lVert w\rVert}=\frac{1}{\lVert w\rVert}$，也就是相当于最小化$\lVert w\rVert^2$。现在的优化问题变为：

$$\begin{align}\displaystyle\operatorname*{min}_{w,b}&\quad\frac{1}{2}\lVert w\rVert^2\\\mathrm{s.t.}&\quad y^{(i)}\left(w^Tx^{(i)}+b\right)\geq 1,\quad i=1,\cdots,m\end{align}$$

这样，我们就把原来棘手的问题变为了可以用软件高效解决的问题。上式是一个带有线性约束的凸二次型问题，解该式即可得到**最优间隔分类器（optimal margin classifier）**。这一类优化问题可以直接使用一些现成的商用二次优化程序（QP: quadratic programming）解决。

到这里，问题基本解决，我们暂停一下支持向量机的讲解，先了解一下拉格朗日对偶。它引出我们优化问题的对偶形式，通过使用核方法，将保证在我们将最优间隔分类器问题推广到极高维度空间时算法的高效求解。对偶形式也将帮助我们推导出一个用以解决上面优化问题的高效算法，其求解速度比常见的QP软件快很多。

## 5. 拉格朗日对偶（Lagrange duality）

我们先放一放支持向量机和最大间隔分类器，来聊一聊带有约束条件的优化问题。考虑下面的问题：

$$\begin{align}\displaystyle\operatorname*{min}_{w}&\quad f(w)\\\mathrm{s.t.}&\quad h_i(w)=0,\quad i=1,\cdots,l\end{align}$$

回忆在高等数学中学到的[拉格朗日乘数法（Lagrange multiplier）](https://en.wikipedia.org/wiki/Lagrange_multiplier)，我们定义**拉格朗日算子（Lagrangian）**：

$$\mathcal{L}(w,\beta)=f(w)+\sum_{i=1}^l\beta_ih_i(w)$$

此处的系数$\beta$称作**拉格朗日乘数（Lagrange multiplier）**，然后将$\mathcal{L}$的偏导数置为零：

$$\begin{align}\frac{\partial\mathcal{L}}{\partial w_i}&=0\\\frac{\partial\mathcal{L}}{\partial\beta_i}&=0\end{align}$$

最后解出$w,\beta$即可。直接参考[拉格朗日乘数法的例题](https://en.wikipedia.org/wiki/Lagrange_multiplier#Examples)更加直观。

在这一节我们会一般化这种带有约束条件的优化问题，也就是不限于等式约束条件，后面会加入不等式约束条件。因为篇幅限制，我们不会在这里直接推导拉格朗日对偶理论（感兴趣可以参考R.T. Rockarfeller (1970), Convex Analysis, Princeton University Press.），仅在这里给出主要思路和结果，进而在应用在最优间隔分类器优化问题中。

下面介绍**原始优化问题（primal optimization problem）**：

$$\begin{align}\displaystyle\operatorname*{min}_{w}&\quad f(w)\\\mathrm{s.t.}&\quad g_i(w)\leq 0,\quad i=1,\cdots ,k\\&\quad h_i(w)=0,\quad i=1,\cdots,l\end{align}$$

定义**广义拉格朗日算子（generalized Lagrangian）**：

$$\mathcal{L}(w,\alpha,\beta)=f(w)+\sum_{i=1}^k\alpha_ig_i(w)+\sum_{i=1}^l\beta_ih_i(w)$$

这里的$\alpha,\beta$是拉格朗日乘数，考虑下面的量：

$$\theta_{\mathcal{P}}(w)=\operatorname*{max}_{\alpha,\beta:\alpha_i\geq 0}\mathcal{L}(w,\alpha,\beta)$$

其中下标$\mathcal{P}$代表"primal"。对于一些$w$，如果$w$违反了任何原始约束条件（如$g_i(w)\gt 0$或$h_i(w)\neq 0$），则可以证明：

$$\begin{align}\theta_{\mathcal{P}}(w)&=\operatorname*{max}_{\alpha,\beta:\alpha_i\geq 0}f(w)+\sum_{i=1}^k\alpha_ig_i(w)+\sum_{i=1}^l\beta_ih_i(w)\\&=\infty\end{align}$$

（如果某个$g_i(w)\gt 0$，则拉格朗日算子中该项相应的拉格朗日乘数$\alpha_i$只需取无穷大就可以使整个式子最大化；类似的，如果某个$h_i(w)\neq 0$，则拉格朗日算子中该项相应的拉格朗日乘数$\beta_i$只需取无穷大也可以使整个式子最大化。）

相反，如果$w$在约束条件内，则有$\theta_{\mathcal{P}}=f(w)$，于是有：

$$\theta_{\mathcal{P}}=\begin{cases}f(w)&\textrm{if }w\textrm{ satisfies primal constraints}\\\infty&\textrm{otherwise}\end{cases}$$

（也就是说，对于满足约束条件的$w$，要使得拉格朗日算子最大化，需要拉格朗日乘数项之和为零。）

则对于满足原始约束的$w$来说，$\theta_{\mathcal{P}}$与原始优化问题中的目标函数相同；对于违反原始约束的$w$来说，$\theta_{\mathcal{P}}$为正无穷。因此，如果考虑最小化：

$$\operatorname*{min}_{w}\theta_{\mathcal{P}}(w)=\operatorname*{min}_{w}\operatorname*{max}_{\alpha,\beta:\alpha_i\geq 0}\mathcal{L}(w,\alpha,\beta)$$

我们发现，这与原始优化问题是一样的（也有同样的解）。为了后面使用方便，我们定义优化目标的最优值为$p^*=\displaystyle\operatorname*{min}_{w}\theta_{\mathcal{P}}(w)$，称为原始问题的**最优值（optimal value）**。

现在，我们来看一个略微不同的问题，定义关于拉格朗日乘数的函数：

$$\theta_{\mathcal{D}}(\alpha,\beta)=\operatorname*{min}_{w}\mathcal{L}(w,\alpha,\beta)$$

其中下标$\mathcal{D}$代表“dual”。留意到在$\theta_{\mathcal{P}}$的定义中，我们优化（最大化）关于$\alpha,\beta$的函数；而在这里，我们最小化关于$w$的函数。

引入**对偶优化问题（dual optimization problem）**：

$$\operatorname*{max}_{\alpha,\beta:\alpha_i\geq 0}\theta_{\mathcal{D}}(\alpha,\beta)=\operatorname*{max}_{\alpha,\beta:\alpha_i\geq 0}\operatorname*{min}_{w}\mathcal{L}(w,\alpha,\beta)$$

这个式子除了“max”和“min”的顺序发生改变以外，其余的同前面的原始问题一样。同样的，定义对偶优化问题的最优值为$d^*=\displaystyle\operatorname*{max}_{\alpha,\beta:\alpha_i\geq 0}\theta_{\mathcal{D}}(\alpha,\beta)$。

原始问题与对偶问题的关系可以用下面的式子简单表示：

$$d^*=\operatorname*{max}_{\alpha,\beta:\alpha_i\geq 0}\operatorname*{min}_{w}\mathcal{L}(w,\alpha,\beta)\leq p^*=\operatorname*{min}_{w}\operatorname*{max}_{\alpha,\beta:\alpha_i\geq 0}\mathcal{L}(w,\alpha,\beta)$$